# Peak Places

Have you ever wondered what the weather looks like at the highest point in each of the 50 states? Then Peak Places is right for you. Peak Places uses a list of the geographic coordinates (latitude and longitude) of the highest point in each state to retrieve location data from the Google Maps suite of APIs and weather data from the Dark Sky API. This initial list is Data from these two APIs is stored in a database titled *peaks* which contains two tables:

* _peaks_information_ for the time invariant Google Maps data
* _peaks_weather_ for the time variant DarkSky weather data

The _peaks_information_ table is only updated once upon the first running of the Peak Places program while the _peaks_weather_ table is updated with new weather data every four hours. Data is never deleted from either table.

### Elevation Scraper
The elevation_scraper.py file is used to generate the states.csv file whihc contains th initial list of geographic coordinates mentioned in the paragraph above. The coordinates are scraped from a Wikipedia article title ["List of U.S. states by elevation"](https://en.wikipedia.org/wiki/List_of_U.S._states_by_elevation) skipping entries for "District of Columbia" and "United States".

In [ ]:
!sudo python3 elevation_scraper.py

### Google Maps API
Peak Places uses a Python Client for Google Maps Services which can be installed using the following command:

In [ ]:
!sudo pip3 install -U googlemaps

The Python Client can query any of the Google Maps APIs using one key. The user must enable each API associated with the key from the [Google API Console](https://developers.google.com/console). For more information on the Google Maps API go here: https://developers.google.com/maps/ and for more information on the Python Client go here: https://github.com/googlemaps/google-maps-services-python.

### DarkSky Weather API
The DarkSky Weather API is used by Peak Places to collect up to the minute weather data at each geographic coordinate every four hours. For more information on the DarkSky API go here: https://darksky.net/dev.

### User IP
The IP for my database connection is 34.226.52.95

What follows is the code for Peak Places, all necessary documentation is commented out in the code:

In [ ]:
# list of imports foe peak_places
import csv
import googlemaps
import requests
import MySQLdb as mdb

from datetime import datetime
from urllib.error import URLError

In [ ]:
# global API key vairables
MAPS_API_KEY = open('GoogleMaps_API_Key.txt', 'r').read()
WEATHER_API_KEY = open('DarkSky_API_Key.txt', 'r').read()

In [ ]:
def get_weather(location):
    '''Collects weather data at geographic coordinate using DarkSky API'''

    # unpack location tupel
    lat = location[0]
    lon = location[1]
    # format url for DarkSky API request
    url = 'https://api.darksky.net/forecast/{API_KEY}/{lat},{lon}'.format(API_KEY=WEATHER_API_KEY,
                                                                          lat=lat, lon=lon)

    # request data
    try:
        r = requests.get(url)
    except:
        return 0
    else:
        weather_json = r.json()['currently']

        # extract pertinent information from API into dictionary
        weather_dict = {
            'lat':        lat,
            'lon':        lon,
            'time':       datetime.fromtimestamp(weather_json['time']),
            'summary':    weather_json['summary'],
            'temp':       weather_json['temperature'],
            'rainProb':   weather_json['precipProbability'],
            'humidity':   weather_json['humidity'],
            'windSpeed':  weather_json['windSpeed'],
            'cloudCover': weather_json['cloudCover'],
            'visibility': weather_json['visibility']
        }

        return weather_dict

In [ ]:
class USMountain(object):
    '''Mountain object populated with data from suite of Google
    Maps APIs'''
    def __init__(self, location):
        self.client = googlemaps.Client(MAPS_API_KEY)
        self.location = location
        self.lat = location[0]
        self.lon = location[1]
        self.__googleStateData()
        self.__googlePlaceData()
        self.__googleElevationData()

    def __googleStateData(self):
        '''Collects state data from Google Geocode API'''
        geocode_json = self.client.reverse_geocode(self.location)

        # assume first result
        geocode_data = geocode_json[0]
        # unpack results from json object returned by Geocode API
        for component in geocode_data['address_components']:
            if 'administrative_area_level_1' in component['types']:
                self.state = component['long_name']

    def __googlePlaceData(self):
        '''Collects place data from Google Places API'''
        places_json = self.client.places_nearby(location=self.location, radius=100,
                                                keyword='mountain', language='en-AU',
                                                type='nature_feature')

        # unpack results from json object returned by Places API
        places_data = places_json['results']

        # check if results were returned before variable assignment
        if places_data:
            # assume first result is correct
            result = places_data[0]
            # place  data in mountains dict
            self.name = result['name']
            self.place_id = result['place_id']
            # in case there is no rating set key to None
            try:
                self.rating = result['rating']
            except:
                self.rating = None
        else:
            # this code is for states SC and FL which do not reutrn anything with above params
            places_json = self.client.places_nearby(location=self.location, radius=500,
                                                    language='en-AU', type='point_of_interest')
            result = places_json['results'][0]
            # place  data in mountains dict
            self.name = result['name']
            self.place_id = result['place_id']
            # in case there is no rating set key to None
            try:
                self.rating = result['rating']
            except:
                self.rating = None

    def __googleElevationData(self):
        '''Collects elevation data from Google Elevation API'''
        elevation_data = self.client.elevation(self.location)

        # unpack results form list object reutrned by Elevation API
        result = elevation_data[0]
        self.elevation = result['elevation']

In [ ]:
# set-up parameters for local host
params = {
    'SERVER': '34.226.52.95',    # whatever the local host is
    'UID':    'root',            # your username
    'PWD':    'dwdstudent2015'   # your password
}
# connect to MySQL db using MySQLdb module
cnxn = mdb.connect(params['SERVER'], params['UID'], params['PWD'],
                   charset='utf8', use_unicode=True)
# cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
# create cursor object
cursor = cnxn.cursor()

# create peaks database if it does not exist
dbname = 'peaks'
create_peaks_database_query = '''CREATE DATABASE IF NOT EXISTS {db}'''.format(db=dbname)
cursor.execute(create_peaks_database_query)

In [ ]:
# create table for peak information if it doesn't exist
create_peak_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table}
                            (place_id varchar(255),
                             name varchar(255),
                             state varchar(255),
                             lat float,
                             lon float,
                             elevation float,
                             rating float,
                             PRIMARY KEY(lat, lon)
                             )'''.format(db=dbname, table='peaks_information')
cursor.execute(create_peak_table_query)

In [ ]:
# create table for weather information if it doesn't exist
create_weather_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table}
                                (lat float,
                                 lon float,
                                 time datetime,
                                 summary varchar(255),
                                 temp float,
                                 rainProb float,
                                 humidity float,
                                 windSpeed float, 
                                 cloudCover float,
                                 visibility float,
                                 PRIMARY KEY(lat, lon, time)
                                 )'''.format(db=dbname, table='peak_weather')
cursor.execute(create_weather_table_query)

In [ ]:
# iterate through longitude and latitude data from states.csv file
with open('states.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # unpack coordinates as a tuple
        location = (float(row['lat']), float(row['lon']))

        # check if peak_information table is filled, if not continue filling
        existence_query = '''SELECT COUNT(*) FROM {db}.{table}'''.format(db=dbname,
                                                                         table='peaks_information')
        cursor.execute(existence_query)
        num_rows = cursor.fetchone()[0]
        if num_rows < 50:
            # create USMountain object
            mountain = USMountain(location)

            # fill table with attributes of mountain object
            peak_info_query = '''INSERT IGNORE INTO {db}.{table}
                                 (place_id, name, state, lat, lon, elevation, rating)
                                 VALUES (%s, %s, %s, %s, %s, %s, %s)'''.format(db=dbname, table='peaks_information')
            query_params = (mountain.place_id, mountain.name, mountain.state, mountain.lat,
                            mountain.lon, mountain.elevation, mountain.rating)
            print(query_params)
            cursor.execute(peak_info_query, query_params)
            cnxn.commit()

        # get weather data
        weather = get_weather(location)
        if weather == 0:
            raise URLError('DarkSky API is down')

        # fill table with weather data
        weather_info_query = '''INSERT IGNORE INTO {db}.{table}
                                (lat, lon, time, summary, temp, rainProb, humidity, windSpeed, cloudCover, visibility)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=dbname, table='peak_weather')
        query_params = (weather['lat'], weather['lon'], weather['time'], weather['summary'],
                        weather['temp'], weather['rainProb'], weather['humidity'],
                        weather['windSpeed'], weather['cloudCover'], weather['visibility'])
        print(query_params)
        cursor.execute(weather_info_query, query_params)
        cnxn.commit()
# close cursor
cursor.close()